In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.loaders import Sentinel2OdcLoader
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsWriter

from pystac import Item

from src.run_task import GeoMADSentinel2Processor, get_grid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask(threads_per_worker=64, n_workers=1, mem_safety_margin="2GB")
client.dashboard_link

In [ ]:
# Issues
# 46,19  # 404 when loading https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/58/K/BE/2023/03/30/S2B_MSIL2A_20230330T232849_N0509_R044_T58KBE_20230331T023901.SAFE/GRANULE/L2A_T58KBE_A031673_20230330T232845/IMG_DATA/R20m/T58KBE_20230330T232849_SCL_20m.tif
# 110,23
# 110,24  # 404 when loading https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/07/L/DD/2023/01/08/S2A_MSIL2A_20230108T194651_N0400_R099_T07LDD_20230109T093105.SAFE/GRANULE/L2A_T07LDD_A039421_20230108T194819/IMG_DATA/R20m/T07LDD_20230108T194651_SCL_20m.tif
# 111,23
# 111,24

# Study site configuration
item_id = "63,20"  # West viti levu
# item_id = "65,22"  # Near the anti-meridian
# item_id = "66,22"  # Right of the anti-meridian
datetime = "2023"

# And get the study site
grid = get_grid()
cell = grid.loc[[(item_id)]]
cell.explore()

In [ ]:
# Set up a data loader
loader = Sentinel2OdcLoader(
    epsg=3832,
    datetime="2023",
    dask_chunksize=dict(time=1, x=4096, y=4096),
    odc_load_kwargs=dict(
        fail_on_error=False,
        resolution=100,
        # bands=["SCL", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
        bands=["SCL", "B04", "B03", "B02"],
        groupby="solar_day",
    ),
    nodata_value=0,
    keep_ints=True,
    load_as_dataset=True,
)

# Run the load process, which is lazy-loaded
input_data = loader.load(cell)
input_data

In [ ]:
# Set up a data processor
processor = GeoMADSentinel2Processor(
    scale_and_offset=False,
    harmonize_to_old=True,
    geomad_options=dict(
        work_chunks=(601, 601),
        num_threads=10,
        maxiters=100,
    ),
    filters=[("closing", 5), ("opening", 5)],
    keep_ints=True
)

# Plan the processing. Still lazy-loaded
output_data = processor.process(input_data)
output_data

In [ ]:
output_data.B04.plot.imshow()

In [ ]:
output_data.odc.to_rgba(bands=["B04", "B03", "B02"], vmin=0, vmax=3000).odc.explore()

In [ ]:
import xarray as xr

xr.__version__

In [ ]:
# Actually load data and do the processing, so we have it in memory
loaded = output_data.compute()
loaded

In [ ]:
loaded["count"].plot.imshow(size=8, robust=True)

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="s2",
    dataset_id="geomad",
    version="0.0.0",
    time=datetime,
)

# Set up a writer and write out the files
writer = LocalDsWriter(
    itempath=dep_path,
    output_nodata=0,
    use_odc_writer=True,
    overwrite=True,
    convert_to_int16=False
)
print(f"Writing to: {writer.itempath}")
out_files = writer.write(loaded, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()

In [ ]:
# Testing the Azure writer

# from dep_tools.writers import AzureDsWriter
# from dep_tools.namers import DepItemPath

# itempath = DepItemPath("geomad", "test", "0.0", datetime)

# writer = AzureDsWriter(
#     itempath=itempath,
#     overwrite=True,
#     convert_to_int16=False,
#     extra_attrs=dict(dep_version="0.0"),
# )

# writer.write(output_data, "test")


In [ ]:
# from odc.stac import load
# from pystac import Item

# # How to make this dynamic?
# stac_url = "https://deppcpublicstorage.blob.core.windows.net/output/dep_geomad_test/0-0/test/2023-01/dep_geomad_test_test_2023-01.stac-item.json"

# item = Item.from_file()

# data = load([item], chunks={})
# data